# Hashing: Secure Hash Algorithm (SHA)

To understand the properties of one-way hash functions, we would like to do the following exercise for SHA256.

1. Create a text file of any length M1.
3. Flip one bit of the input file (not one byte!) into M2. You can achieve this modification using ghex or bless or `python`.
2. Generate the hash values H1 of M1 and H2 of M2.
5. Please observe whether H1 and H2 are similar or not.

Please refer to https://cryptography.io/en/latest/hazmat/primitives/cryptographic-hashes/ for details of the usage of cryptographic-hashes.

In [1]:
# visualization tools
import textwrap

color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=True, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

def visual_hex_diff(bstr_1, bstr_2, hex_names=("HEX 1", "HEX 2")):
    SEP = "   |   "
    print("  ", f"{hex_names[0]}".ljust(16 + 7), hex_names[1], sep=SEP)
    # block level
    hex_1, hex_2 = textwrap.wrap(bstr_1.hex(), 16), textwrap.wrap(bstr_2.hex(), 16)
    for i, (block_1, block_2) in enumerate(zip(hex_1, hex_2)):
        # byte level
        block_1, block_2 = textwrap.wrap(block_1, 2), textwrap.wrap(block_2, 2)
        block_2 = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(block_1, block_2)]
        print(str(i).rjust(2), " ".join(block_1).ljust(16 + 7), " ".join(block_2).ljust( 16 + 7), sep=SEP)

In [2]:
M1 = b"0123"

In [3]:
import random

random.seed(0)  # Set the seed to 0 to get a predictable output

# Convert bytes string to a list of integers
int_list = list(M1)

# Generate a random index to select an integer from the list
index = random.randint(0, len(int_list) - 1)

# Generate a random bit position to flip (0 to 7)
bit_position = random.randint(0, 7)

# Flip the bit at the random position in the selected integer
original_int = int_list[index]
int_list[index] ^= (1 << bit_position)

# Convert the list of integers back to a bytes string
M2 = bytes(int_list)
print(M2)

b'012s'


In [4]:
# visualize the bit flipping
original_bits = list(bin(original_int)[2:].rjust(8, "0"))
flipped_bits = list(bin(int_list[index])[2:].rjust(8, "0"))
flipped_bits = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(original_bits, flipped_bits)]
print("Before Flipping: \t{} ('{}', 0x{})".format(" ".join(original_bits), M1.decode()[index], M1.decode()[-1].encode().hex()))
print("After Flipping: \t{} ('{}', 0x{})".format(" ".join(flipped_bits), M2.decode()[index], M2.decode()[-1].encode().hex()))

Before Flipping: 	0 0 1 1 0 0 1 1 ('3', 0x33)
After Flipping: 	0 1 1 1 0 0 1 1 ('s', 0x73)


In [5]:
visual_hex_diff(M1, M2, ("M1", "M2"))

     |   M1                        |   M2
 0   |   30 31 32 33               |   30 31 32 73


In [6]:
from cryptography.hazmat.primitives import hashes

def digest(msg):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(msg)
    return digest.finalize()

H1, H2 = digest(M1), digest(M2)
visual_hex_diff(H1, H2, ("H1", "H2"))

     |   H1                        |   H2
 0   |   1b e2 e4 52 b4 6d 7a 0d   |   f6 67 75 65 aa ba 82 19
 1   |   96 56 bb b1 f7 68 e8 24   |   50 07 f8 d9 13 28 f8 04
 2   |   8e ba 1b 75 ba ed 65 f5   |   fa 6b 96 eb 6c e9 91 be
 3   |   d9 9e af a9 48 89 9a 6a   |   4b d0 97 cd e6 d9 d7 71


# MD5 Collision

In [7]:
!/usr/bin/md5collgen

MD5 collision generator v1.5
by Marc Stevens (http://www.win.tue.nl/hashclash/)

Allowed options:
  -h [ --help ]           Show options.
  -q [ --quiet ]          Be less verbose.
  -i [ --ihv ] arg        Use specified initial value. Default is MD5 initial 
                          value.
  -p [ --prefixfile ] arg Calculate initial value using given prefixfile. Also 
                          copies data to output files.
  -o [ --out ] arg        Set output filenames. This must be the last option 
                          and exactly 2 filenames must be specified. 
                          Default: -o msg1.bin msg2.bin



In [8]:
!/usr/bin/md5collgen -o exp1_msg1.bin exp1_msg2.bin

MD5 collision generator v1.5
by Marc Stevens (http://www.win.tue.nl/hashclash/)

Using output filenames: 'exp1_msg1.bin' and 'exp1_msg2.bin'
Using initial value: 0123456789abcdeffedcba9876543210

Generating first block: ...
Generating second block: S10...........
Running time: 4.10271 s


In [9]:
with open("exp1_msg1.bin", "rb") as f:
    exp1_msg1 = f.read()

In [10]:
exp1_msg1

b'8L\xe8*\x88\x01Z\xd7\xcf$\xe8\xa3\x18\xb7]e\x02\xb6\xee\x90\x0ck\x1dnRn9\x0e\xc9\x8c\x88\xf4\xf2\xc1\xaf\xea~\x0f*:\x8bhV\xf4\xf8\xcf6F\xf0\xb2\xcd\xd9.A\xbe\x17{\xe2Q\x8b`\xb3GeJB5#\xffp\xc4\xab\xe92\x13,:\x01\xac\x16\x1f\xd2\x97nX\x99\xa1\xef\xab\xb3\x00\xcf;`H\xc7\xd3\r\x0c\x18\x97I\xff5v=\xccZ+\x04"I9\xef\xfbz\x87\r\xd6\x82q\x86\xfe\xe7\xac\x19\t\x18'

In [11]:
with open("exp1_msg2.bin", "rb") as f:
    exp1_msg2 = f.read()

In [12]:
exp1_msg2

b'8L\xe8*\x88\x01Z\xd7\xcf$\xe8\xa3\x18\xb7]e\x02\xb6\xee\x10\x0ck\x1dnRn9\x0e\xc9\x8c\x88\xf4\xf2\xc1\xaf\xea~\x0f*:\x8bhV\xf4\xf8O7F\xf0\xb2\xcd\xd9.A\xbe\x17{\xe2Q\x0b`\xb3GeJB5#\xffp\xc4\xab\xe92\x13,:\x01\xac\x16\x1f\xd2\x97\xeeX\x99\xa1\xef\xab\xb3\x00\xcf;`H\xc7\xd3\r\x0c\x18\x97I\xff5v=\xccZ+\x84!I9\xef\xfbz\x87\r\xd6\x82q\x86\xfeg\xac\x19\t\x18'

In [13]:
visual_hex_diff(exp1_msg1, exp1_msg2, ("exp1_msg1", "exp1_msg2"))

     |   exp1_msg1                 |   exp1_msg2
 0   |   38 4c e8 2a 88 01 5a d7   |   38 4c e8 2a 88 01 5a d7
 1   |   cf 24 e8 a3 18 b7 5d 65   |   cf 24 e8 a3 18 b7 5d 65
 2   |   02 b6 ee 90 0c 6b 1d 6e   |   02 b6 ee 10 0c 6b 1d 6e
 3   |   52 6e 39 0e c9 8c 88 f4   |   52 6e 39 0e c9 8c 88 f4
 4   |   f2 c1 af ea 7e 0f 2a 3a   |   f2 c1 af ea 7e 0f 2a 3a
 5   |   8b 68 56 f4 f8 cf 36 46   |   8b 68 56 f4 f8 4f 37 46
 6   |   f0 b2 cd d9 2e 41 be 17   |   f0 b2 cd d9 2e 41 be 17
 7   |   7b e2 51 8b 60 b3 47 65   |   7b e2 51 0b 60 b3 47 65
 8   |   4a 42 35 23 ff 70 c4 ab   |   4a 42 35 23 ff 70 c4 ab
 9   |   e9 32 13 2c 3a 01 ac 16   |   e9 32 13 2c 3a 01 ac 16
10   |   1f d2 97 6e 58 99 a1 ef   |   1f d2 97 ee 58 99 a1 ef
11   |   ab b3 00 cf 3b 60 48 c7   |   ab b3 00 cf 3b 60 48 c7
12   |   d3 0d 0c 18 97 49 ff 35   |   d3 0d 0c 18 97 49 ff 35
13   |   76 3d cc 5a 2b 04 22 49   |   76 3d cc 5a 2b 84 21 49
14   |   39 ef fb 7a 87 0d d6 82   |   39 ef fb 7a 87 0d d6 82
15   |

In [14]:
prefix = "Hello, World!"
with open("prefix.txt", "w") as f:
    f.write(prefix)

In [15]:
!/usr/bin/md5collgen -o exp2_msg1.bin exp2_msg2.bin -p prefix.txt

MD5 collision generator v1.5
by Marc Stevens (http://www.win.tue.nl/hashclash/)

Using output filenames: 'exp2_msg1.bin' and 'exp2_msg2.bin'
Using prefixfile: 'prefix.txt'
Using initial value: 7b0db9355979517113266afbb9e2f08e

Generating first block: .....
Generating second block: S11.........
Running time: 3.62773 s


In [16]:
with open("exp2_msg1.bin", "rb") as f:
    exp2_msg1 = f.read()
with open("exp2_msg2.bin", "rb") as f:
    exp2_msg2 = f.read()

In [17]:
prefix.encode().hex()

'48656c6c6f2c20576f726c6421'

In [18]:
visual_hex_diff(exp2_msg1, exp2_msg2, ("exp2_msg1", "exp2_msg2"))

     |   exp2_msg1                 |   exp2_msg2
 0   |   48 65 6c 6c 6f 2c 20 57   |   48 65 6c 6c 6f 2c 20 57
 1   |   6f 72 6c 64 21 00 00 00   |   6f 72 6c 64 21 00 00 00
 2   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 3   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 4   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 5   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 6   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 7   |   00 00 00 00 00 00 00 00   |   00 00 00 00 00 00 00 00
 8   |   80 08 2c de d0 e4 87 12   |   80 08 2c de d0 e4 87 12
 9   |   d5 42 71 7a 05 7c 34 b7   |   d5 42 71 7a 05 7c 34 b7
10   |   a5 40 dc 5a f5 bf 06 f4   |   a5 40 dc da f5 bf 06 f4
11   |   36 9e 7b 5e 0e aa b8 d0   |   36 9e 7b 5e 0e aa b8 d0
12   |   35 9d b0 e7 5e 30 2a 63   |   35 9d b0 e7 5e 30 2a 63
13   |   0b 24 6f d3 a8 01 4c 3c   |   0b 24 6f d3 a8 81 4c 3c
14   |   15 ff 47 a4 6e 30 5e 8c   |   15 ff 47 a4 6e 30 5e 8c
15   |

In [19]:
import random

random.seed(0)  # Set the seed to 0 to get a predictable output
bytes(random.randrange(256) for _ in range(16)).hex()

'c5d71484f8cf9bf4b76f47904730804b'

In [20]:
!/usr/bin/md5collgen -o exp3_msg1.bin exp3_msg2.bin -i a1b8c8e9072dd1d8ca7938bc0f65fc4c

MD5 collision generator v1.5
by Marc Stevens (http://www.win.tue.nl/hashclash/)

Using output filenames: 'exp3_msg1.bin' and 'exp3_msg2.bin'
Using initial value: a1b8c8e9072dd1d8ca7938bc0f65fc4c

Generating first block: ...................................
Generating second block: S10.........
Running time: 30.8252 s


In [21]:
with open("exp3_msg1.bin", "rb") as f:
    exp3_msg1 = f.read()
with open("exp3_msg2.bin", "rb") as f:
    exp3_msg2 = f.read()
visual_hex_diff(exp3_msg1, exp3_msg2, ("exp3_msg1", "exp3_msg2"))

     |   exp3_msg1                 |   exp3_msg2
 0   |   01 e6 3e 13 74 e0 0b 1c   |   01 e6 3e 13 74 e0 0b 1c
 1   |   91 7e 86 89 f8 aa 21 be   |   91 7e 86 89 f8 aa 21 be
 2   |   0e 4c 87 25 7a 3d 5a 2a   |   0e 4c 87 a5 7a 3d 5a 2a
 3   |   26 ae 7c 6a 30 b4 b9 15   |   26 ae 7c 6a 30 b4 b9 15
 4   |   91 db be e4 38 85 86 f2   |   91 db be e4 38 85 86 f2
 5   |   07 04 6c d2 9c 57 64 1b   |   07 04 6c d2 9c d7 64 1b
 6   |   21 64 55 f3 82 ec db f7   |   21 64 55 f3 82 ec db f7
 7   |   76 af d3 a5 4a 8f 02 33   |   76 af d3 25 4a 8f 02 33
 8   |   44 95 26 ba 4f b6 b2 65   |   44 95 26 ba 4f b6 b2 65
 9   |   89 31 e2 cf a2 7b cd 51   |   89 31 e2 cf a2 7b cd 51
10   |   31 74 56 8b 74 5f c3 39   |   31 74 56 0b 74 5f c3 39
11   |   8d c5 20 58 ab 5f 88 c8   |   8d c5 20 58 ab 5f 88 c8
12   |   cf b3 24 38 af 84 2e 34   |   cf b3 24 38 af 84 2e 34
13   |   b4 3b cc 7a d9 ab fd 4c   |   b4 3b cc 7a d9 2b fd 4c
14   |   0b 46 a2 3c c9 2e e1 04   |   0b 46 a2 3c c9 2e e1 04
15   |